In [ ]:
_exp_name = 0

In [1]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm

from sklearn.model_selection import KFold
import torchvision.models as models
import Aentations as A
from Aentations.pytorch import ToTensorV2
import cv2
from torch.autograd import Variable

In [2]:
myseed = 459  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [ ]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = A.Compose([
    A.Resize(256, 256),
    A.CenterCrop(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = A.Compose([
    A.Resize(256, 256), # fit resnet34
    A.CenterCrop(224, 224),
    A.OneOf([
        A.Blur(p = 0.2),
        A.GaussianBlur(p = 0.2),
        A.MedianBlur(p = 0.2),
    ]),
    album.ShiftScaleRotate(p = 0.3),
    album.RGBShift(p = 0.4),
    album.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [ ]:
class FoodDataset(Dataset):

    def __init__(self,path,train_tfm = train_tfm, 
    test_tfm = test_tfm,files = None, mode = 'train'):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.train_transform = train_tfm
        self.test_transform = test_tfm
        self.mode = mode
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = cv2.imread(fname)
        
        #im = self.data[idx]

        # ref: https://github.com/Aentations-team/Aentations
        train_im = self.train_transform(image=im)['image']
        try:
            label = int(fname.split("\\")[-1].split("_")[0])
        except:
            label = -1 # test has no label

        if self.mode == 'train':
            return train_im, label
        else:
            test_im = self.test_transform(image=im)['image']
            train_ims = [train_im]
            for i in range(4):
                train_ims.append(self.train_transform(image=im)['image'])
            return train_ims, test_im, label



In [ ]:
batch_size = 64
_dataset_dir = "./food11"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset(os.path.join(_dataset_dir,"training"), mode='train')
valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), mode='train')
concat = ConcatDataset([train_set, valid_set])


In [ ]:
#ref: https://github.com/facebookresearch/mixup-cifar10/blob/main/train.py
def mixup_data(x, y, alpha = 0.5):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [ ]:
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
split = 4
n_epochs = 100
patience = 100

folds = KFold(n_splits=split, shuffle=True)

for fold, (train_ids, test_ids) in enumerate(folds.split(concat)):
    train = torch.utils.data.SubsetRandomSampler(train_ids)
    valid = torch.utils.data.SubsetRandomSampler(test_ids)
    train_loader = DataLoader(concat,batch_size = batch_size, shuffle = False, sampler = train, pin_memory = True)
    valid_loader = DataLoader(concat,batch_size = batch_size, shuffle = False, sampler = valid, pin_memory = True)

    model = models.resnet34(pretrained = False, num_classes=5).to(device)
    # model.apply(ResetWeight)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.0005)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 10)

    best_accuracy = 0
    
    for epoch in range(n_epochs):
        model.train()
        train_accuracy = []
        train_loss = []

        for batch in train_loader:
            imgs, labels = batch
            imgs, labels = imgs.cuda(), labels.cuda()

            inputs, targets_a, targets_b, lam = mixup_data(imgs, labels, alpha = 0.4)
            inputs, targets_a, targets_b = map(Variable, (inputs, targets_a, targets_b))

            logits = model(inputs)
            loss = mixup_criterion(criterion, logits, targets_a, targets_b, lam)
            optimizer.zero_grad()
            loss.backward()

            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5.0)
            optimizer.step()

            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            train_loss.append(loss.item())
            train_accuracy.append(acc)
        
        scheduler.step()
        train_acc = sum(train_accuracy) / len(train_accuracy)
        train_loss = sum(train_loss) / len(train_loss)

        print(f"Fold: {fold} [ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
        model.eval()
        
        valid_accuracy = []
        valid_loss = []

        for batch in valid_loader:
            test_im, labels = batch
            with torch.no_grad():
                logits = model(test_im.to(device))
            loss = criterion(logits, labels.to(device))
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            valid_loss.append(loss.item())
            valid_accuracy.append(acc)
        
        valid_acc = sum(valid_accuracy) / len(valid_accuracy)
        valid_loss = sum(valid_loss) / len(valid_loss)
        print(f"Fold: {fold} [ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        if valid_acc > best_accuracy:
            print(f"Best model found at epoch {epoch}, saving model")
            torch.save(model.state_dict(), f"./models/{_exp_name}_fold_{fold}.ckpt")
            best_accuracy = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break

            

In [ ]:
import gc
del train_loader, valid_loader
gc.collect()

In [ ]:
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), mode='test')
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle = False, pin_memory = True)

In [ ]:
def TTA(model, train_ims, test_im):
    test_pred = model(test_im.to(device))
    train_pred = []
    for im in train_ims:
        train_pred.append(model(im.to(device)))

    avg = sum(train_pred) / len(train_pred)
    return avg * 0.5 + test_pred * 0.5


In [ ]:
predict = []
for fold in range(split):
    pred = []
    best_model = models.resnet34(pretrained = False, num_classes = 5).to(device)
    best_model.load_state_dict(torch.load(f"./models/{_exp_name}_fold_{fold}.ckpt"))
    best_model.eval()
    with torch.no_grad():
        for train_im, test_im, i in test_loader:
            test_pred = TTA(best_model, train_im, test_im)
            pred.append(test_pred)
    predict.append(torch.cat(pred))

mixpred = torch.stack(predict, dim = 0).sum(dim = 0) / split

In [ ]:
prediction = []
test_label = np.argmax(mixpred.cpu().data.numpy(), axis = 1)
prediction += test_label.squeeze().tolist()

In [ ]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
name = 'predict.csv'
df.to_csv(name,index = False)